Analyze 90° AFM exchange interactions: are the specific ligands of this edge also part of other edges with AFM interactions and large bond angles?  

As AFM exchange of two half-filled d orbitals usually higher in magnitude as various FM exchange interactions, violating an FM 90° bond angle in favor of a AFM large bond angle may be energetically favorable. *Please note*: this analysis is very short and local - in principle, enumeration of KGA-abiding and -violating edges across all likely magnetic structures would be necessary here.

In [1]:
from collections import Counter
import json
from monty.json import MontyDecoder
import numpy as np
import os
import pandas as pd
from pymatgen.core import Element
import plotly.express as px

from utils_kga.general import pretty_plot

In [2]:
# Load edge-df
with open("data/dfs_of_magnetic_edge_information.json") as f:
    dict_all_stats = json.load(f)
all_stats = {key: pd.DataFrame.from_dict(df) for key, df in dict_all_stats.items()}
all_stats = {key: df.drop(columns=[c for c in df.columns if c.endswith("clude_ligand_multiplicities")]) for key, df in all_stats.items()}
with open("../../data_retrieval_and_preprocessing_MAGNDATA/data/df_grouped_and_chosen_commensurate_MAGNDATA.json") as f:
    df = json.load(f, cls=MontyDecoder)

In [3]:
# Add is_tm bool for later easier analysis
for ang_df in all_stats.values():
    ang_df["site_is_tm"] = ang_df["site_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["site_to_is_tm"] = ang_df["site_to_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["ligand_el_set"] = ang_df["ligand_elements"].apply(lambda ls: set(ls))
    ang_df["av_bond_angle"] = ang_df["mag_ligand_mag_angle"].apply(lambda ls: np.mean(np.array(ls)))
    ang_df["spin_angle"] = ang_df["spin_angle"].apply(lambda x: round(x, -1))  # same subset as in bond angle analysis

In [4]:
plot_dir = "plots/TM_octahedra_analysis_KGA_frustration"
os.makedirs(plot_dir, exist_ok=True)
description = [
    "all edges with TM octahedra at both nodes",
    "all oxygen edges with TM octahedra at both nodes",
]
large_bond_angle_lower_bound = 110

In [5]:
all_data = {}
for bond_angle_range90 in [[90.0, 90.0], [89.9, 90.1], [89.5, 90.5], [89, 91], [88, 92], [85, 95], [80, 100]]:
    data = {data_string: {} for data_string in description}
    for data_string in description:
        kga_true_ligand_fraction = {}
        for overall_id, (md_id, ang_df) in enumerate(all_stats.items()):
            subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
                   & (ang_df["site_to_ce"]=="O:6")
                   & (ang_df["site_is_tm"]) 
                   & (ang_df["site_to_is_tm"])
            ]
            if "oxygen" in data_string:
                subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
            if not subdf.empty:
                afm_df = subdf.loc[subdf["spin_angle"]>=170.0]
                if not afm_df.empty:
                    ligand_dict = {"L_in_90_AFM": set(), "L_in_90_AFM_and_larger_bond_angle": set()}
                    for afm_row_id, afm_row in afm_df.iterrows():
                        for sub_edge_idx, ligand_idx in enumerate(afm_row["ligand_indices"]):
                            if bond_angle_range90[0] <= afm_row["mag_ligand_mag_angle"][sub_edge_idx] <= bond_angle_range90[1]:
                                ligand_dict["L_in_90_AFM"].add(ligand_idx)
                                for afm_row_id2, afm_row2 in afm_df.iterrows():  # only check for KGA subset right now
                                    if ligand_idx in afm_row2["ligand_indices"]:
                                        idx_of_ligand_idx = np.argwhere(np.array(afm_row2["ligand_indices"])==ligand_idx)[0][0]
                                        if afm_row2["mag_ligand_mag_angle"][idx_of_ligand_idx] > large_bond_angle_lower_bound:
                                            ligand_dict["L_in_90_AFM_and_larger_bond_angle"].add(ligand_idx)
                    
                    if len(ligand_dict["L_in_90_AFM"]) > 0:
                        kga_true_ligand_fraction[md_id] = len(ligand_dict["L_in_90_AFM_and_larger_bond_angle"]) / len(ligand_dict["L_in_90_AFM"]) if len(ligand_dict["L_in_90_AFM"]) > 0 else 0
                            
        data[data_string] = kga_true_ligand_fraction
    all_data[tuple(bond_angle_range90)] = data
for ba_range, data in all_data.items():
    print(ba_range)
    for datastring, subset in data.items():
        print(datastring)
        print(Counter(list(subset.values())))

(90.0, 90.0)
all edges with TM octahedra at both nodes
Counter({1.0: 6, 0.0: 1})
all oxygen edges with TM octahedra at both nodes
Counter({1.0: 2})
(89.9, 90.1)
all edges with TM octahedra at both nodes
Counter({1.0: 9, 0.0: 2})
all oxygen edges with TM octahedra at both nodes
Counter({1.0: 5})
(89.5, 90.5)
all edges with TM octahedra at both nodes
Counter({1.0: 12, 0.0: 5})
all oxygen edges with TM octahedra at both nodes
Counter({1.0: 7, 0.0: 2})
(89, 91)
all edges with TM octahedra at both nodes
Counter({1.0: 17, 0.0: 13})
all oxygen edges with TM octahedra at both nodes
Counter({1.0: 12, 0.0: 8})
(88, 92)
all edges with TM octahedra at both nodes
Counter({0.0: 21, 1.0: 19})
all oxygen edges with TM octahedra at both nodes
Counter({0.0: 14, 1.0: 13})
(85, 95)
all edges with TM octahedra at both nodes
Counter({0.0: 42, 1.0: 20, 0.5: 2})
all oxygen edges with TM octahedra at both nodes
Counter({0.0: 32, 1.0: 14, 0.5: 2})
(80, 100)
all edges with TM octahedra at both nodes
Counter({0.0

In [12]:
# Plot as stacked bar plot
for data_string in description:
    names = ["KGA frustration percentage", "n structures", "bond angle interval"]
    dat = []
    for ba_range, data in all_data.items():
        c = Counter(list(data[data_string].values()))
        sorted_keys = sorted(list(c.keys()))
        c = {k: c[k] for k in sorted_keys}
        for perc, n_struct in c.items():
            dat.append([perc, n_struct, str(ba_range)])
    df_plot = pd.DataFrame(dat, columns=names)
    fig = px.bar(df_plot, x="bond angle interval", y="n structures", color="KGA frustration percentage", title=f"KGA frustration percentage {data_string}", color_continuous_scale="viridis")
    fig = pretty_plot(fig)
    fig.update_layout(font=dict(size=22))
    fig.show()
    fig.write_image(os.path.join(plot_dir, f"kga_frustration_percentage_{data_string}.pdf"))

### Notes on experiment temperatures of subset close to 90°
General issue of MAGNDATA: atomic positions may be from below or above magnetic ordering temperature -> especially when analyzing bond angles in a very small interval, ensure the atomic positions are from below magnetic ordering temperature to ensure respective angles are really exactly 90° (with symmetry-forbidden AFM interaction) **in the magnetic phase**

------
*structural data from below transition temperature*?
nmo = not mentioned otherwise
- 0.499 yes (very high Tc) -> frustration fraction = 0.0
- 1.233 nmo
- 1.28 probably yes (high Tc)
- 1.287 nmo
- 1.31 no
- 1.58 nmo
- 1.582 yes, but atomic positions from another study
- 1.6 yes (very high Tc)
- 1.619 nmo
- 1.659 no -> frustration fraction = 0.0
- 1.69 yes (high Tc)